# GPT Fintuning - PROJECT

In [ ]:
# Uncomment the following to install PyTorch and Transformers libraries

#%pip install torch
#%pip install transformers

In [ ]:
import torch
import transformers
print(torch.__version__)
print(transformers.__version__)

Reviewing a pretrained model:

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Set the seed for PyTorch (controls randomness for reproducibility)
seed = 1
torch.manual_seed(seed)

# Load pretrained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

# Encode input context to get token IDs
input_text = "AITA for pretending to get fired when customers get a temper with me?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate text using the model
output = model.generate(input_ids, 
                        do_sample=True, 
                        max_length=150, 
                        repetition_penalty=1.1,
                        temperature=1, 
                        top_k=50, 
                        top_p=0.95
                        )

# Decode the generated IDs to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


🔔 **Question**: Does this output make sense? 

### Visualizing Embeddings 

Change the words in the `words` variable to words you are interested in exploring. 

Think about your own data already: which words  might show up often in the Reddit data you are working with?

In [ ]:
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
embeddings = model.transformer.wte.weight.detach().numpy()

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
words = ["mother", "father", "wife", "husband"]
word_indices = [tokenizer.encode(word)[0] for word in words]
selected_embeddings = embeddings[word_indices]

In [ ]:
selected_embeddings

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Using PCA
pca = PCA(n_components=2)
reduced_embeddings_pca = pca.fit_transform(selected_embeddings)

In [ ]:
# Uncomment the following line to install adjustText 

#%pip install adjustText

In [ ]:
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Function to plot
def plot_embeddings(embeddings, labels, title='PCA of GPT-2 Embeddings'):
    plt.figure(figsize=(8, 6))
    texts = []
    points = plt.scatter(embeddings[:, 0], embeddings[:, 1])  # scatter plot of embeddings

    # Generate text annotations
    for i, label in enumerate(labels):
        x, y = embeddings[i]
        text = plt.text(x, y, label, ha='right', va='bottom', fontsize=9)
        texts.append(text)

    # Use adjust_text to avoid overlapping
    adjust_text(texts, x=embeddings[:, 0], y=embeddings[:, 1], arrowprops=dict(arrowstyle='->', color='red', lw=0.5))

    plt.title(title)
    plt.grid(True)
    plt.show()

# Example usage (assuming 'reduced_embeddings_pca' and 'words' are defined)
plot_embeddings(reduced_embeddings_pca, words, title='PCA of GPT-2 Embeddings')


<a id="ft"></a>

# Finetuning GPT-2 

In all of the cells below, replace YOUR_FILE with the name of the file you are working with.

### ⚠️ Warning
Even though we are only training a small model, the following blocks of code operation will take long on a consumer-grade PC (for reference: around 4 hours on an Apple M2 Pro with 16GB memory). I have run this code and saved the model, so you don't have to run it yourself. Please move on to "Getting the Model From Google Drive". 

If you do choose to run the model yourself, you may want to run this notebook in Google Colab using a GPU. See bCourses for a link.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/YOUR_FILE.csv')

In [ ]:
texts = df['selftext'].tolist()

## Commence Tokenization

We will tokenize the entire texts with truncation and padding to a fixed maximum length. This method is straightforward and treats each text as an individual sequence for the model to learn from. The main characteristics include:

- `truncation`: Texts longer than `max_length=512` are cut off, potentially losing important information at the end.
- `padding`: Texts shorter than `max_length=512` are padded to ensure uniform sequence length, usually with the pad_token. This is not relevant to us as all texts we are feeding into the model are much longer than 512 tokens.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Initialize tokenizer with padding token set
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token

# Tokenize texts
encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

In [ ]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings.input_ids)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # For language modeling, the labels are the input_ids shifted by one
        item["labels"] = item["input_ids"].clone()
        return item

# Initialize the dataset
train_dataset = TextDataset(encodings)

## Commence Finetuning

In the following cell, we initialize the fine-tuning process using Hugging Face's `Trainer` class.

⚠️ **Warning:** Even though we are using a small model, the following operation will take *long* on a consumer-grade PC (for reference: around 4 hours on an Apple M2 Pro with 16GB memory).

In [ ]:
# Initialize the model
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

# Define training arguments
training_args = TrainingArguments(
    output_dir='../../results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='../../logs'
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start training
trainer.train()

If you did run the previous code, make sure to save the model and finetuned tokenizer:

In [ ]:
model_save_path = '../../models/my_finetuned_model'
tokenizer_save_path = '../../models/my_finetuned_tokenizer'

# Save the model
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)


<a id="int"></a>
# Interpreting Model Output

Enter your own prompt using the `prompt` variable. Pick a prompt that resembles posts in your subreddit (e.g. a title, or the first few words of a post)!

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Set the seed for PyTorch (controls randomness for reproducibility)
seed = 1
torch.manual_seed(seed)

def generate_text(model, prompt, do_sample=True, max_length=50, temperature=1, top_k=50, top_p=0.95, repetition_penalty=1.1):
    """
    Generates text based on a given prompt using the specified model.
    
    Parameters:
    - model: The fine-tuned model to use for text generation.
    - prompt: The initial text to start generating from.
    - max_length: Maximum length of the generated text.
    - temperature: Sampling temperature for generating text.
    - top_k: The number of highest probability vocabulary tokens to keep for top-k filtering.
    - top_p: Nucleus sampling's cumulative probability cutoff to keep for top-p filtering.
    
    Returns:
    - generated_text: The generated text as a string.
    """
    # Encode the prompt text to tensor
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    # Generate a sequence of tokens following the prompt
    output_ids = model.generate(input_ids, max_length=max_length, 
                                temperature=temperature, 
                                do_sample=do_sample, 
                                top_k=top_k, 
                                top_p=top_p, 
                                repetition_penalty=repetition_penalty, 
                                pad_token_id=tokenizer.eos_token_id)
    
    # Decode the generated tokens to a string
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return generated_text

# Load the fine-tuned model
ft_model = GPT2LMHeadModel.from_pretrained('../../models/my_finetuned_model')

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('../../models/my_finetuned_tokenizer')

# Prompt to generate text from - play around with this!
prompt = "ENTER YOUR OWN PROMPT HERE"

# Generate texts
generated_text = generate_text(ft_model, prompt, max_length=150)
print("Generated text from finetuned model:", generated_text, '\n')


## Visualizing Finetuned Embeddings

Let's look at the embedding space.

Change the words in the `words` variable to words you are interested in exploring. 


In [ ]:
ft_model = GPT2LMHeadModel.from_pretrained('../../models/my_finetuned_model')
embeddings = ft_model.transformer.wte.weight.detach().numpy()

tokenizer = GPT2Tokenizer.from_pretrained('../../models/my_finetuned_tokenizer')
words = ["mother", "father", "wife", "husband"]
word_indices = [tokenizer.encode(word)[0] for word in words]
selected_embeddings = embeddings[word_indices]

In [ ]:
selected_embeddings

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Using PCA
pca = PCA(n_components=2)
reduced_embeddings_pca = pca.fit_transform(selected_embeddings)

In [ ]:
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Function to plot
def plot_embeddings(embeddings, labels, title='PCA of GPT-2 Embeddings'):
    plt.figure(figsize=(8, 6))
    texts = []
    points = plt.scatter(embeddings[:, 0], embeddings[:, 1])  # scatter plot of embeddings

    # Generate text annotations
    for i, label in enumerate(labels):
        x, y = embeddings[i]
        text = plt.text(x, y, label, ha='right', va='bottom', fontsize=9)
        texts.append(text)

    # Use adjust_text to avoid overlapping
    adjust_text(texts, x=embeddings[:, 0], y=embeddings[:, 1], arrowprops=dict(arrowstyle='->', color='red', lw=0.5))

    plt.title(title)
    plt.grid(True)
    plt.show()

# Example usage (assuming 'reduced_embeddings_pca' and 'words' are defined)
plot_embeddings(reduced_embeddings_pca, words, title='PCA of Finetuned GPT-2 Embeddings')


# Create Posts Using Finetuned Model

We will ask the model to generate text based on the title of the original posts in your data. We will also ask it to create a post that is the same length as the original post.

In [ ]:
import pandas as pd
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

# Load your data
df = pd.read_csv('../../data/YOUR_FILE.csv')

# Randomly select 10 entries
sample_df = df.sample(n=10, random_state=1)  # Use a fixed seed for reproducibility, if needed

original_texts = sample_df['selftext'].tolist()  # Adjust column name if different
titles = sample_df['title'].tolist()

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('../../models/my_finetuned_tokenizer')
model = GPT2LMHeadModel.from_pretrained('../../models/my_finetuned_model')

# Initialize the generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1)  # CPU usage

# Prepare to generate texts
generated_texts = []
for title, original_text in zip(titles, original_texts):
    # Calculate the length of the original post in tokens
    target_length = len(tokenizer.encode(original_text))

    # Generate a new post of the same length starting from the title
    # Ensure to set max_length to the length of the original post
    prompt = title
    generated = generator(prompt, max_length=target_length, num_return_sequences=1)[0]['generated_text']
    generated_texts.append(generated)

Let's save the original and generated posts in a new DataFrame so we can easily compare them.

In [ ]:
df = pd.DataFrame({
    'original_text': original_texts,
    'generated_text': generated_texts,
    'title': titles  # Assuming you have a list of titles
})

Let's compare original posts to the ones we created with GPT-2. Here we are selecting an index from our new DataFrame and printing the text for both the original and our generated text.

In [ ]:
index = 2
print(f"Title: {df.loc[index]['title']}")
print("\n")
print(f"Original Text: {df.loc[index, 'original_text']}")
print("\n")
print(f"Generated Text: {df.loc[index, 'generated_text']}")

In [ ]:
# %pip install TextBlob

In [ ]:
from textblob import TextBlob

def compare_posts(index):
    if index < 0 or index >= len(df):
        print("Invalid index, please choose a valid post index.")
        return
    
    # Display the texts
    print(f"Title: {df.loc[index, 'title']}")

    # Sentiment analysis
    original_blob = TextBlob(df.loc[index, 'original_text'])
    generated_blob = TextBlob(df.loc[index, 'generated_text'])
    print("\nSentiment Analysis:")
    print(f"Original Sentiment: Polarity = {original_blob.sentiment.polarity}, Subjectivity = {original_blob.sentiment.subjectivity}")
    print(f"Generated Sentiment: Polarity = {generated_blob.sentiment.polarity}, Subjectivity = {generated_blob.sentiment.subjectivity}")
    
# Example usage: compare post at index 5
compare_posts(5)


In [ ]:
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt

# Define a function to calculate sentiment polarity
def get_sentiment_polarity(texts):
    return [TextBlob(text).sentiment.polarity for text in texts]

# Calculate sentiment polarity for both original and generated texts
df['original_polarity'] = get_sentiment_polarity(df['original_text'])
df['generated_polarity'] = get_sentiment_polarity(df['generated_text'])

plt.figure(figsize=(10, 6))
plt.scatter(df['original_polarity'], df['generated_polarity'], alpha=0.5, color='blue')
plt.title('Comparison of Sentiment Polarity between Original and Generated Texts')
plt.xlabel('Original Text Sentiment Polarity')
plt.ylabel('Generated Text Sentiment Polarity')
plt.grid(True)
plt.axhline(y=0, color='gray', linestyle='--')
plt.axvline(x=0, color='gray', linestyle='--')
plt.show()


## Back to TF-IDF

Use the TF-IDF algorithm to check the similarity between original texts and the ones we generated:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine original and generated texts into one list for TF-IDF analysis
texts = df['original_text'].tolist() + df['generated_text'].tolist()

# Initialize a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the data
tfidf_matrix = vectorizer.fit_transform(texts)

# Calculate cosine similarity between original and generated texts
# Assuming the first half are originals and the second half are generated
similarity_matrix = cosine_similarity(tfidf_matrix[:len(df)], tfidf_matrix[len(df):])

# Display similarity results
for i, similarity in enumerate(similarity_matrix.diagonal()):
    print(f"Text {i+1} Similarity between original and generated: {similarity:.4f}")


## Back to Word Embeddings

Project the reduced embeddings for both the original and our generated posts in 2D space:

In [ ]:
import spacy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Load the medium model with word vectors
nlp = spacy.load("en_core_web_sm")

# Sample data (assuming df is already loaded)
original_texts = df['original_text'].tolist()[:10]  # Limiting for simplicity
generated_texts = df['generated_text'].tolist()[:10]

# Function to compute average embeddings for a text
def get_average_embedding(text, nlp_model):
    doc = nlp_model(text)
    vectors = [word.vector for word in doc if not word.is_stop and word.has_vector]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros((nlp_model.vocab.vectors_length,))

# Compute average embeddings for each set of texts
original_embeddings = np.array([get_average_embedding(text, nlp) for text in original_texts])
generated_embeddings = np.array([get_average_embedding(text, nlp) for text in generated_texts])

# Perform PCA to reduce dimensionality for visualization
pca = PCA(n_components=2)
original_pca = pca.fit_transform(original_embeddings)
generated_pca = pca.transform(generated_embeddings)

# Plotting the embeddings
plt.figure(figsize=(10, 8))
plt.scatter(original_pca[:, 0], original_pca[:, 1], color='blue', label='Original', alpha=0.5)
plt.scatter(generated_pca[:, 0], generated_pca[:, 1], color='red', label='Generated', alpha=0.5)
plt.title('Comparison of Word Embeddings')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid(True)
plt.show()
